In [ ]:
import warnings
warnings.filterwarnings("ignore")
from pathlib import Path
import geopandas as gp
import pandas as pd
from pyproj import Transformer, CRS, Proj
from shapely.geometry import shape, Point, Polygon, box
from matplotlib.colors import LinearSegmentedColormap
import numpy as np
import shapely.speedups as speedups
import contextily as ctx
from collections import Counter
import matplotlib.pyplot as plt
import mapclassify as mc
speedups.enable()
import re
from python_hll.hll import HLL
import mmh3
from python_hll.util import NumberUtil
import emoji
import operator

In [ ]:
# Define some constants to be used throughout the program

#create grids based on the study area shapefile

GRID_SIZE_METERS = 100000
                        
# target projection: Mollweide
EPSG_CODE = 54009
CRS_PROJ = f"esri:{EPSG_CODE}"

# Input projection WGS 84
CRS_WGS = "epsg:4326"

# define Transformer to project things to Mollweide
PROJ_TRANSFORMER = Transformer.from_crs(
    CRS_WGS, CRS_PROJ, always_xy=True)

# also define reverse projection
PROJ_TRANSFORMER_BACK = Transformer.from_crs(
    CRS_PROJ, CRS_WGS, always_xy=True)

#projecting the bounds of the study area shapefile to Mollweide
XMIN = PROJ_TRANSFORMER.transform(-18.729512 , 29.234046)[0]
XMAX = PROJ_TRANSFORMER.transform(39.73858, 29.234046)[0]
YMAX = PROJ_TRANSFORMER.transform(49.59352369, 71.16987838)[1]
YMIN = PROJ_TRANSFORMER.transform(49.59352369, 28.017169)[1]

In [ ]:
# read in the study area shapefile
europe = gp.read_file("Europe_Clipped_BBox.shp")
europe.to_crs(CRS_PROJ, inplace =True)
europe.plot()

In [ ]:
europe.head()

In [ ]:
def create_grids():
    
#     Creating polygons based on the grid size
    
    width = GRID_SIZE_METERS
    length = GRID_SIZE_METERS
    cols = list(range(int(np.floor(XMIN)), int(np.ceil(XMAX)), width))
    rows = list(range(int(np.floor(YMIN)), int(np.ceil(YMAX)), length))
    rows.reverse()

    polygons = []
    for x in cols:
         for y in rows:
                # combine to tuple: (x,y, poly)
                # and append to list
                polygons.append(
                    (x, y,
                     Polygon([
                         (x, y),
                         (x+width, y),
                         (x+width, y-length),
                         (x, y-length)])))
    grid = pd.DataFrame(polygons)
        # name columns
    col_labels=['xbin', 'ybin', 'bin_poly']
    grid.columns = col_labels
        # use x and y as index columns
    grid.set_index(['xbin', 'ybin'], inplace=True)
    grid = gp.GeoDataFrame(
            grid.drop(
                columns=["bin_poly"]),
                geometry=grid.bin_poly)
    grid.crs = CRS_PROJ
    return grid,cols,rows

grid,cols,rows = create_grids()

In [ ]:
centroid_grid = grid.centroid.reset_index()
centroid_grid.set_index(["xbin", "ybin"], inplace=True)

In [ ]:
grid.centroid

In [ ]:
from geopandas.tools import sjoin
def intersect_grid_centroids(
    grid: gp.GeoDataFrame, 
    intersect_gdf: gp.GeoDataFrame):
    """Return grid centroids from grid that 
    intersect with intersect_gdf
    """
    centroid_grid = gp.GeoDataFrame(
        grid.centroid)
    centroid_grid.rename(
        columns={0:'geometry'},
        inplace=True)
    centroid_grid.set_geometry(
        'geometry', crs=grid.crs, 
        inplace=True)
    grid_intersect = sjoin(
        centroid_grid, intersect_gdf, 
        how='right')
    grid_intersect.set_index(
        ["index_left0", "index_left1"],
        inplace=True)
    grid_intersect.index.names = ['xbin','ybin']
    return grid.loc[grid_intersect.index]

In [ ]:
grid.boundary.plot()

#### There are several different aggregation levels avaialble to use with HLL data depending on how much we would like to reduce spatial accuracy and therefore preserve user privacy. Let's plot a few to see how the aggregation levels differ:

In [ ]:
# open and read csv file containing HLL data aggregated to level 5
df5 = pd.read_csv(r"C:\Users\saman\OneDrive\Documents\Thesis\Data\HLL_Data_Final_5.csv")
# Convert to geodataframe, projection WGS84
gdf5 = gp.GeoDataFrame(df5,geometry =gp.points_from_xy(df5.longitude_5,df5.latitude_5),crs =4326)
gdf5.to_crs(CRS_PROJ,inplace=True)


In [ ]:
# plot the points on top of study area
fig, ax = plt.subplots(figsize=(35, 20))
grid.plot(ax=ax, color='white', edgecolor='black', linewidth=0.1)
europe.boundary.plot(ax=ax, color="black", linewidth=0.8)
gdf5.plot(ax=ax, color="purple", markersize=3)
ax.set_title("Twitter Post Locations (Aggregated to Level 5)", fontsize=20)
ax.set_axis_off()
plt.show()
fig.savefig(r"C:\Users\saman\OneDrive\Documents\Thesis\Figures\AllPosts_mapped_5_grid.png", dpi=300, bbox_inches = "tight")

In [ ]:
# open and read csv file containing HLL data aggregated to level 4
df4 = pd.read_csv(r"C:\Users\saman\OneDrive\Documents\Thesis\Data\HLL_Data_Final_4.csv")
# Convert to geodataframe, projection WGS84
gdf4 = gp.GeoDataFrame(df4,geometry=gp.points_from_xy(df4.longitude_4,df4.latitude_4),crs =4326)
gdf4.to_crs(CRS_PROJ,inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(35, 20))
grid.plot(ax=ax, color='white', edgecolor='black', linewidth=0.1)
europe.boundary.plot(ax=ax, color="black", linewidth=0.8)
gdf4.plot(ax=ax, color="purple", markersize=3)
ax.set_title("Twitter Post Locations (Aggregated to Level 4)", fontsize=20)
ax.set_axis_off()
plt.show()
fig.savefig(r"C:\Users\saman\OneDrive\Documents\Thesis\Figures\AllPosts_mapped_4_grid.png", dpi=300, bbox_inches = "tight")

In [ ]:
# open and read csv file containing HLL data aggregated to level 3
df3 = pd.read_csv (r"C:\Users\saman\OneDrive\Documents\Thesis\Data\HLL_Data_Final_3.csv")
# Convert to geodataframe, projection WGS84
gdf3 = gp.GeoDataFrame(df3,geometry =gp.points_from_xy(df3.longitude_3,df3.latitude_3),crs =4326)
# project gdf to Mollweide
gdf3.to_crs(CRS_PROJ,inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(35, 20))
grid.plot(ax=ax, color='white', edgecolor='black', linewidth=0.1)
europe.boundary.plot(ax=ax, color="black", linewidth=0.8)
gdf3.plot(ax=ax, color="purple", markersize=10)
ax.set_title("Twitter Post Locations (Aggregated to Level 3)", fontsize=20)
ax.set_axis_off()
plt.show()
fig.savefig(r"C:\Users\saman\OneDrive\Documents\Thesis\Figures\AllPosts_mapped_3_grid.png", dpi=300, bbox_inches = "tight")

#### It looks like an aggregation level of 4 is a good balance between spatial accuracy and privacy preservation. An aggregation level of 5 seems to not have a significant benefit for privacy and level 3 poses issues where the data has such a coarse resolution that it creates some blank grid cells. 

In [ ]:
gdf = gdf4
# rename lat and long columns to avoid confusion/simplify things
gdf.rename(columns = {'latitude_4':'latitude', 'longitude_4':'longitude'}, inplace = True)
gdf.head()

In [ ]:
# before starting our analysis, let's double-check to make sure every row has an emoji 
# (no blank rows due to flags rendering incorrectly, etc.)
gdf_noemoji = gdf[gdf['emoji'] =='']
gdf_noemoji.head()

In [ ]:
# method to join HLL post info with these grid cells using np.digitize
ybins = np.array(rows)
xbins = np.array(cols)

def get_best_bins(search_values_x, search_values_y,xbins, ybins): 
    """Will return best bin for a lat and lng input
    
    Note: prepare bins and values in correct matching projection
    
    """
    xbins_idx = np.digitize(search_values_x, xbins, right=False)
    ybins_idx = np.digitize(search_values_y, ybins, right=False)
    return (xbins[xbins_idx-1], ybins[ybins_idx-1])


xbins_match, ybins_match = get_best_bins(
    search_values_x=gdf.geometry.x.to_numpy(),
    search_values_y=gdf.geometry.y.to_numpy(),
    xbins=xbins, ybins=ybins)

In [ ]:
gdf.loc[:, 'xbins_match'] = xbins_match
gdf.loc[:, 'ybins_match'] = ybins_match
gdf.set_index(['xbins_match', 'ybins_match'], inplace=True)
grid.sort_index(inplace =True)
gdf.sort_index(inplace = True)
common_idx = grid.index.intersection(gdf.index) 
#instead of a spatial join, indexes are used to find which hashtag belongs to which grid
gdf

In [ ]:
def hll_from_byte(hll_set: str):
# Return HLL set from binary representation
    hex_string = hll_set[2:]
    return HLL.from_bytes(
        NumberUtil.from_hex(
            hex_string, 0, len(hex_string)))
def cardinality_from_hll(hll_set):
# Turn binary hll into HLL set and return cardinality
    hll = hll_from_byte(hll_set)
    return hll.cardinality() - 1

In [ ]:
def union_hll(hll: HLL, hll2):
    """Union of two HLL sets. The first HLL set will be modified in-place."""
    hll.union(hll2)
    
def union_all_hll(
    hll_series: pd.Series, cardinality: bool = True) -> pd.Series:
    """HLL Union and (optional) cardinality estimation from series of hll sets

        Args:
        hll_series: Indexed series (bins) of hll sets. 
        cardinality: If True, returns cardinality (counts). Otherwise,
            the unioned hll set will be returned.
    """
    
    hll_set = None
    for hll_set_str in hll_series.values.tolist():
        if hll_set is None:
            # set first hll set
            hll_set = hll_from_byte(hll_set_str)
            continue
        hll_set2 = hll_from_byte(hll_set_str)
        union_hll(hll_set, hll_set2)
    return hll_set.cardinality()

In [ ]:
gdf.head()

In [ ]:
#NEW

In [ ]:
def grid_userdays(new_test,idx):    
        counter = Counter(
        n_s = union_all_hll(gdf.loc[idx,'pud_hll'].dropna())
        ud.loc[idx,'pud_hll'] = n_s

ud = pd.DataFrame(index = common_idx, columns = ['pud_hll'], data = '') #dummy dataframe to hold the typicality values

for idx,midx in enumerate(common_idx): #looping through all the common indexes between the grids and dataframe
    grid_userdays(gdf.loc[midx,"pud_hll"], common_idx[idx])

geom = grid.loc[common_idx, "geometry"]
ud_gdf = gp.GeoDataFrame(data = ud['pud_hll'], geometry =geom, crs = CRS_PROJ)
ud_gdf

In [ ]:
bins = 500, 2000, 5000, 10000, 200000
base = grid.plot(figsize=(22,28), color='white', edgecolor='black', linewidth=0.1)
plot = ud_gdf.plot(ax=base, colormap='Purples', column='pud_hll', alpha = 0.7, edgecolor='gray', linewidth=0.1, legend=True,
                   scheme='UserDefined',
                   classification_kwds={'bins': bins})
# plot.legend(prop={'size': 6})
europe.boundary.plot(ax=base, edgecolor='dimgray', linewidth=0.7,)
plt.title("Distribution of User Days", size =35)
fig = plot.get_figure()
fig.savefig(r"C:\Users\saman\OneDrive\Documents\Thesis\Figures\Total_User_Days.png", dpi=300, bbox_inches = "tight")

### Theoretically, we could calculate the number of userdays for every country present in the dataset. This is unfortunately too computationally intensive and time consuming for the scope of this project, so total userdays were only calculated for a subset of countries based on their coverage with the available dataset 

In [ ]:
# create empty dictionary to fill with countries and their total userdays
country_ud = {}

In [ ]:
uk = europe[europe['NAME_EN'] == 'United Kingdom']
# make country grid
grid_uk = intersect_grid_centroids(
    grid=grid, intersect_gdf=uk)
# join hll data to grid
uk_hll = gdf.sjoin(grid_uk, how="right")
# join together country grid cells, calculate the number of userdays
# within the cluster of grid cells
userdays_uk = union_all_hll(uk_hll["pud_hll"].dropna())
# add the result to a dictionary
country_ud["United Kingdom"] = userdays_uk

In [ ]:
fr = europe[europe['NAME_EN'] == 'France']
grid_fr = intersect_grid_centroids(
    grid=grid, intersect_gdf=fr)
fr_hll = gdf.sjoin(grid_fr, how="right")
userdays_fr = union_all_hll(fr_hll["pud_hll"].dropna())
country_ud["France"] = userdays_fr

In [ ]:
sp = europe[europe['NAME_EN'] == 'Spain']
grid_sp = intersect_grid_centroids(
    grid=grid, intersect_gdf=sp)
sp_hll = gdf.sjoin(grid_sp, how="right")
userdays_sp = union_all_hll(sp_hll["pud_hll"].dropna())
country_ud["Spain"] = userdays_sp

In [ ]:
it = europe[europe['NAME_EN'] == 'Italy']
grid_it = intersect_grid_centroids(
    grid=grid, intersect_gdf=it)
it_hll = gdf.sjoin(grid_it, how="right")
userdays_it = union_all_hll(it_hll["pud_hll"].dropna())
country_ud["Italy"] = userdays_it

In [ ]:
de = europe[europe['NAME_EN'] == 'Germany']
grid_de = intersect_grid_centroids(
    grid=grid, intersect_gdf=de)
de_hll = gdf.sjoin(grid_de, how="right")
userdays_de = union_all_hll(de_hll["pud_hll"].dropna())
country_ud["Germany"] = userdays_de

In [ ]:
ne = europe[europe['NAME_EN'] == 'Netherlands']
grid_ne = intersect_grid_centroids(
    grid=grid, intersect_gdf=ne)
ne_hll = gdf.sjoin(grid_ne, how="right")
userdays_ne = union_all_hll(ne_hll["pud_hll"].dropna())
country_ud["Netherlands"] = userdays_ne

In [ ]:
tur = europe[europe['NAME_EN'] == 'Turkey']
grid_tur = intersect_grid_centroids(
    grid=grid, intersect_gdf=tur)
tur_hll = gdf.sjoin(grid_tur, how="right")
userdays_tur = union_all_hll(tur_hll["pud_hll"].dropna())
country_ud["Turkey"] = userdays_tur

In [ ]:
cz = europe[europe['NAME_EN'] == 'Czech Republic']
grid_cz = intersect_grid_centroids(
    grid=grid, intersect_gdf=cz)
cz_hll = gdf.sjoin(grid_cz, how="right")
userdays_cz = union_all_hll(cz_hll["pud_hll"].dropna())
country_ud["Czech Republic"] = userdays_cz

In [ ]:
be = europe[europe['NAME_EN'] == 'Belgium']
grid_be = intersect_grid_centroids(
    grid=grid, intersect_gdf=be)
be_hll = gdf.sjoin(grid_be, how="right")
userdays_be = union_all_hll(be_hll["pud_hll"].dropna())
country_ud["Belgium"] = userdays_be

In [ ]:
sw = europe[europe['NAME_EN'] == 'Switzerland']
grid_sw = intersect_grid_centroids(
    grid=grid, intersect_gdf=sw)
sw_hll = gdf.sjoin(grid_sw, how="right")
userdays_sw = union_all_hll(sw_hll["pud_hll"].dropna())
country_ud["Switzerland"] = userdays_sw

In [ ]:
po = europe[europe['NAME_EN'] == 'Portugal']
grid_po = intersect_grid_centroids(
    grid=grid, intersect_gdf=po)
po_hll = gdf.sjoin(grid_po, how="right")
userdays_po = union_all_hll(po_hll["pud_hll"].dropna())
userdays_po
country_ud["Portugal"] = userdays_po

In [ ]:
au = europe[europe['NAME_EN'] == 'Austria']
grid_au = intersect_grid_centroids(
    grid=grid, intersect_gdf=au)
au_hll = gdf.sjoin(grid_au, how="right")
userdays_au = union_all_hll(au_hll["pud_hll"].dropna())
country_ud["Austria"] = userdays_au

In [ ]:
"""
This dictionary is the result of the above code when run on HLL data with an aggregation level of 4.
"""
country_ud

In [ ]:
"""
The following dictionary was the result of calculating the cardinality of userdays per country while using an aggregation 
level of 5. Between these and the previous results, the error rate of HLL (3-5%) is clearly illustrated.
"""
country_ud 

In [ ]:
country_ud_sorted = sorted(country_ud.items(), key=operator.itemgetter(1), reverse=True)
country_ud_sorted

top 10 countries by userdays are:

United Kingdom
Spain
France
Germany
Italy
Turkey
Netherlands
Belgium
Switzerland
Austria

In [ ]:
# let's also calculate the total number of userdays across the entire study area
total_userdays = union_all_hll(gdf["pud_hll"].dropna())
total_userdays

## Experiment using unions to combine emojis with different skin tones 

In [ ]:
thumbsupgdf = gdf[gdf['emoji'].str.contains('👍|👍🏻|👍🏼|👍🏽|👍🏾|👍🏿')]
thumbsupgdf.head()

In [ ]:
# let's see if we can now find userdays per emoji
thumbsup_ud = union_all_hll(thumbsupgdf["pud_hll"].dropna())
thumbsup_ud

In [ ]:
# let's compare that value for the number of userdays for just the generic (yellow) thumbs up emoji
thumbsup_gen_gdf = gdf[gdf['emoji']=='👍']
thumbsup_gen_ud = union_all_hll(thumbsup_gen_gdf["pud_hll"].dropna())
thumbsup_gen_ud

In [ ]:
# great - it works! now I'll repeat these for all skin-tone emojis in the list of top 50 most frequently ocurring emojis

In [ ]:
prayhandsgdf = gdf[gdf['emoji'].str.contains('🙏|🙏🏻|🙏🏼|🙏🏽|🙏🏾|🙏🏿')]
prayhands_ud = union_all_hll(prayhandsgdf["pud_hll"].dropna())
prayhands_ud

In [ ]:
clapgdf = gdf[gdf['emoji'].str.contains('👏|👏🏻|👏🏼|👏🏽|👏🏾|👏🏿')]
clap_ud = union_all_hll(clapgdf["pud_hll"].dropna())
clap_ud

In [ ]:
# make dictionary of emoji names and their variations
emojidictemo = {
    ":clapping_hands:": "👏|👏🏻|👏🏼|👏🏽|👏🏾|👏🏿",
    ":folded_hands:": "🙏|🙏🏻|🙏🏼|🙏🏽|🙏🏾|🙏🏿",
    ":thumbs_up:": "👍|👍🏻|👍🏼|👍🏽|👍🏾|👍🏿",
    ":flexed_biceps:": "💪|💪🏻|💪🏼|💪🏽|💪🏾|💪🏿",
    ":OK_hand:": "👌|👌🏻|👌🏼|👌🏽|👌🏾|👌🏿",
    ":raising_hands:": "🙌|🙌🏻|🙌🏼|🙌🏽|🙌🏾|🙌🏿",
    ":backhand_index_pointing_down:": "👇|👇🏻|👇🏼|👇🏽|👇🏾|👇🏿",
    ":backhand_index_pointing_right:": "👉|👉🏻|👉🏼|👉🏽|👉🏾|👉🏿",
    ":victory_hand:": "✌️|✌🏻|✌🏼|✌🏽|✌🏾|✌🏿",
    ":oncoming_fist:": "👊|👊🏻|👊🏼|👊🏽|👊🏾|👊🏿"
}

emoji_ud = {}

for name, variations in emojidictemo.items():
    subset = gdf[gdf['emoji'].str.contains(variations)]
    emoji_ud[name] = union_all_hll(subset["pud_hll"].dropna())

emoji_ud

In [ ]:
# now let's run it for the top 50 most common emojis by absolute frequency and compare user days to post count
# there should be a huge difference for emojis primarily used by bots


In [ ]:
topemojisdf = gdf['emoji'].value_counts()
top100list = topemojisdf.head(100)
top100dict = top100list.to_dict()

emoji_userdays_100 = {}
top100 = top100dict.keys()
for emoj in top100:
    genericemoji = emoji.demojize(emoj).replace("_light_skin_tone","").replace("_medium-light_skin_tone","").replace("_medium_skin_tone","").replace("_medium-dark_skin_tone","").replace("_dark_skin_tone","")
    if genericemoji in emojidictemo.keys():
        emo = emojidictemo[genericemoji]
    else:
        emo = emoji.emojize(genericemoji)
    emokey = emoji.emojize(genericemoji) # this should avoid any long lists in the dictionary keys
    subset = gdf[gdf['emoji'].str.contains(emo)]
    emoji_userdays_100[emoj] = union_all_hll(subset["pud_hll"].dropna())

emoji_userdays_100    

In [ ]:
emoji_userdays_100 = {'❤️': 187938, '😍': 130963, '🔴': 34552, '😂': 190683, '😎': 58073, '☀️': 44965, '😊': 69271, 
                      '💙': 82002, '😉': 65919, '🥰': 59354, '🔥': 66638, '👍': 95117, '🤣': 82473, '😁': 54824, 
                      '🚓': 2050, '💪': 87392, '❤': 187938, '📸': 33003, '🙏': 93921, '🤩': 35565, '✨': 38818, 
                      '💚': 41228, '🤔': 52586, '👏': 99485, '🎉': 29751, '😋': 27232, '🚗': 3572, '😘': 37036, 
                      '🖤': 37687, '👌': 50397, '🌞': 20908, '💛': 45365, '♥️': 33384, '😅': 35820, '💕': 31422,
                      '🤗': 30618, '🎶': 29636, '💜': 37766, '😀': 27025, '🎄': 20375, '🥳': 24477, '😜': 24255, 
                      '🌊': 14894, '😭': 39637, '🌈': 26916, '⚽️': 40216, '🥂': 15282, '😷': 21079, '✅': 22893, 
                      '👀': 27175, '📷': 15527, '🤪': 20030, '🙄': 31538, '😃': 18466, '🙈': 24852, '💥': 20408, 
                      '😱': 22194, '❄️': 11336, '💖': 19160, '💪🏻': 87392, '☺️': 16571, '🙌': 45654, '🌸': 16146, 
                      '😢': 20462, '🤍': 17627, '🌹': 20112, '🚒': 3050, '🏆': 16975, '🍻': 11991, '🍀': 17220, 
                      '🐶': 12552, '📍': 12179, '🧡': 15547, '😳': 20269, '👉': 26892, '😄': 14377, '💯': 15759, 
                      '💫': 14460, '🍷': 11211, '🚑': 3789, '🙂': 14460, '👑': 14204, '👏🏻': 99485, '➡️': 14741, 
                      '🙏🏻': 93921, '🌿': 11228, '🍾': 9812, '😏': 15820, '👍🏻': 95117, '🌟': 13790, '😬': 15466, 
                      '🌅': 7918, '🎂': 10607, '🌳': 9174, '👇': 36136, '🎁': 10240, '🔝': 11544, '🔵': 15461,
                      '😆': 13585, '🌍': 11352}

In [ ]:
top_emojis_ud = sorted(emoji_userdays_100.items(), key=operator.itemgetter(1), reverse=True)
top_emojis_ud

In [ ]:
emoji_postcount_100 = {}
top100 = top100dict.keys()
for emoj in top100:
    genericemoji = emoji.demojize(emoj).replace("_light_skin_tone","").replace("_medium-light_skin_tone","").replace("_medium_skin_tone","").replace("_medium-dark_skin_tone","").replace("_dark_skin_tone","")
    if genericemoji in emojidictemo.keys():
        emo = emojidictemo[genericemoji]
    else:
        emo = emoji.emojize(genericemoji)
    emokey = emoji.emojize(genericemoji) # this should avoid any long lists in the dictionary keys
    subset = gdf[gdf['emoji'].str.contains(emo)]
    emoji_postcount_100[emoj] = union_all_hll(subset["post_hll"].dropna())

emoji_postcount_100    

In [ ]:
diffdict = {}
for emo in emoji_userdays_100.keys():
    diffdict[emo] = emoji_postcount_100[emo] - emoji_userdays_100[emo]
diffdict_sorted = sorted(diffdict.items(), key=operator.itemgetter(1), reverse=True)
diffdict_sorted